<h1 style="text-align:center;">AF3 prediction with a single TCR chain</h1>

In [2]:
import glob
import os
import pandas as pd

# Infer TCR sequence
(we actually don't need that, Juliens AF3 script does that automatically)

In [45]:
# Construct lookup table
anno_dir = '/Users/roessner/Documents/PostDoc/Data/MixTCRviz/data_raw/HomoSapiens' # full sequences for VJ-segments
ref_csvs = glob.glob(f'{anno_dir}/*.csv')
ref_csvs.sort()

ref_tables = {}

for file in ref_csvs:
    name = os.path.basename(file).replace('.csv', '')  # e.g. TRAV
    df = pd.read_csv(file)
    df['full_clean'] = df['full'].str.replace(r'[-*]', '', regex=True)
    ref_tables[name] = df

lookup = {}

for gene_type, df in ref_tables.items():
    df = df.set_index('Unnamed: 0')
    
    lookup[gene_type] = {
        'full': df['full_clean'].to_dict(),
        'cdr3': df['CDR3'].to_dict()
    }

In [46]:
def reconstruct_chain(v_gene, j_gene, observed_cdr3, v_type, j_type):
    v_full = lookup[v_type]['full'][v_gene]
    v_ref_cdr3 = lookup[v_type]['cdr3'][v_gene]

    j_full = lookup[j_type]['full'][j_gene]
    j_ref_cdr3 = lookup[j_type]['cdr3'][j_gene]

    v_clean = v_full[:-len(v_ref_cdr3)]
    j_clean = j_full[len(j_ref_cdr3):]
    
    return v_clean + observed_cdr3 + j_clean


In [48]:
anno_dir = '/Users/roessner/Documents/PostDoc/Data/MixTCRviz/data_raw/HomoSapiens' # full sequences for VJ-segments

batches = ['LAU5013/YF_LAU5013_sc_WT', 'Public_Data/YF_public_pairedData_20251010']
           
for batch in batches:
    batch_short = batch.split('/')[0]

    # originial data from julien containing V/J-segements + observed CDR3 sequences
    file = f"{batch.split('/')[1]}_AF3.tsv"
    df = pd.read_csv(file, sep='\t')[['TRAV', 'TRAJ', 'TRBV', 'TRBJ', 'cdr3_TRA', 'cdr3_TRB']]
    df = df.dropna()

    df['TRA_full'] = df.apply(
    lambda row: reconstruct_chain(
        row['TRAV'], row['TRAJ'], row['cdr3_TRA'],
        'TRAV', 'TRAJ'
    ),
    axis=1
)

    df['TRB_full'] = df.apply(
        lambda row: reconstruct_chain(
            row['TRBV'], row['TRBJ'], row['cdr3_TRB'],
            'TRBV', 'TRBJ'
        ),
        axis=1
    )

    df.to_csv(f'{batch_short}.csv', index=False)
    

# Prepare AF3 input

In [ ]:
anno_dir = '/Users/roessner/Documents/PostDoc/Data/MixTCRviz/data_raw/HomoSapiens' # full sequences for VJ-segments

batches = ['LAU5013/YF_LAU5013_sc_WT', 'Public_Data/YF_public_pairedData_20251010']
chains = ['A', 'B']     

for batch in batches:
    batch_short = batch.split('/')[0]

    # originial data from julien containing V/J-segements + observed CDR3 sequences
    file = f"{batch.split('/')[1]}_AF3.tsv"
    df = pd.read_csv(file, sep='\t')[['id', 'TRAV', 'TRAJ', 'TRBV', 'TRBJ', 'cdr3_TRA', 'cdr3_TRB']]
    df = df.dropna()
    df['peptide'] = 'LLWNGPMAV'
    df['MHC'] = 'HLA_A0201'
    df['species'] = 'HomoSapiens'
    
    df.to_csv(f'{batch_short}_AF3_input.csv', index=False)

    for chain in chains:
        if chain == 'A':
            df_chain = df.drop(columns=['TRBV', 'TRBJ','cdr3_TRB'])
            df_chain.to_csv(f'{batch_short}_AF3_input_chainA.csv', index=False)
        else:
            df_chain = df.drop(columns=['TRAV', 'TRAJ','cdr3_TRA'])
            df_chain.to_csv(f'{batch_short}_AF3_input_chainB.csv', index=False)

        
    


2042
249
389
378
